In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
movies = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv')

In [ ]:
movies.head()

In [ ]:
movies.shape

In [ ]:
credits.head()

In [ ]:
movie_rate = movies.merge(credits,on='title')

In [ ]:
movie_rate.head(2)

In [ ]:
movie_rate = movie_rate[['movie_id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movie_rate.head(2)

In [ ]:
movie_rate.isnull().sum()

In [ ]:
movie_rate.dropna(inplace=True)

In [ ]:
import ast

In [ ]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L

In [ ]:
movie_rate['genres'][0]

In [ ]:
movie_rate['genres'] = movie_rate['genres'].apply(convert)
movie_rate.head(2)

In [ ]:
movie_rate['keywords'] = movie_rate['keywords'].apply(convert)
movie_rate.head(2)

In [ ]:
#top 3 actors
def convert1(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L

In [ ]:
movie_rate['cast'] = movie_rate['cast'].apply(convert1)
movie_rate.head(2)

In [ ]:
movie_rate['crew'][0]

In [ ]:

def getDirector(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

In [ ]:
movie_rate['crew'] = movie_rate['crew'].apply(getDirector)
movie_rate.head(2)

In [ ]:
movie_rate["overview"][0]

In [ ]:
#to make overview column compatible to other columns
movie_rate['overview'] = movie_rate['overview'].apply(lambda x:x.split())

In [ ]:
movie_rate.head(2)

In [ ]:
#Remove whitespace 
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [ ]:
movie_rate['cast'] = movie_rate['cast'].apply(collapse)
movie_rate['crew'] = movie_rate['crew'].apply(collapse)
movie_rate['genres'] = movie_rate['genres'].apply(collapse)
movie_rate['keywords'] = movie_rate['keywords'].apply(collapse)

In [ ]:
movie_rate["name"]= movie_rate['title'].apply(lambda x: x.replace(" ", ""))

In [ ]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [ ]:
movie_rate['name']=movie_rate["name"].apply(remove_punctuations)

In [ ]:
movie_rate['name'] = movie_rate['name'].apply(lambda x:x.split())

In [ ]:
movie_rate.head(2)

In [ ]:
movie_rate['tags'] = movie_rate['name']+ movie_rate['overview'] + movie_rate['genres'] + movie_rate['keywords'] + movie_rate['cast'] + movie_rate['crew']

In [ ]:
movie_rate.head(2)

In [ ]:
new = movie_rate.drop(columns=['overview','genres','keywords','cast','crew','name'])

In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new.head()

In [ ]:

new['tags'] = new['tags'].apply(remove_punctuations)
new.head(2)

In [ ]:
new['tags'] = new['tags'].apply(lambda x: x.lower())
new.head()

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps =PorterStemmer()
nltk.download('stopwords')
nltk.download('wordnet')
stopword_list = nltk.corpus.stopwords.words('english')

In [ ]:

new['tags'] = new['tags'].apply(lambda x: ' '.join([i for i in x.split() if i not in stopword_list]))
new['tags'] = new['tags'].apply(lambda x:' '.join([ps.stem(i) for i in x.split()]))
new.head()

In [ ]:
new['tags'][0]

# TEXT TO VECTOR

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vector = cv.fit_transform(new['tags']).toarray()

In [ ]:
vector

In [ ]:
vector.shape

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity

In [ ]:
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
recommend('Men in Black')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer().fit_transform(new['tags']).toarray()

In [ ]:
similarity1 = cosine_similarity(tfidf)

In [ ]:
def recommend1(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity1[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

In [ ]:
recommend1('Men in Black')

In [ ]:
import pickle

In [ ]:
new.to_pickle('data.pkl')

In [ ]:
pickle.dump(new,open('movieRating.pkl','wb'))
pickle.dump(similarity1,open('similarityScore.pkl','wb'))